In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skfuzzy as fuzz
import skfuzzy.membership as mf
from skfuzzy import control as ctrl
from scipy.interpolate import interp1d

In [2]:
#exporting dataset
url = "C:\\Users\\YAU JIA YIN\\Desktop\\School\\UTHM\\20232024\\SEM 5\\BIT33703 FUZZY SYSTEM DEVELOPMENT\\Project\\bmi.csv"
df = pd.read_csv(url, header = None, names=['Height', 'Weight'])

In [3]:
#show the data
df

,Height,Weight
0,174,96
1,189,87
2,185,110
3,195,104
4,149,61
...,...,...
495,150,153
496,184,121
497,141,136
498,150,95


In [4]:
#input variables
height = ctrl.Antecedent(np.arange(145, 180, 1), 'height')
weight = ctrl.Antecedent(np.arange(35, 85, 1), 'weight')

#output variable
bmi = ctrl.Consequent(np.arange(15, 31, 0.1), 'bmi')

In [5]:
#membership funtion
height['short'] = mf.trapmf(height.universe, [120, 140, 150, 165])
height['normal'] = mf.trapmf(height.universe, [150, 165, 165, 175])
height['tall'] = mf.trapmf(height.universe, [160, 175, 180, 180])

weight['thin'] = mf.trapmf(weight.universe, [30, 30, 40, 55])
weight['normal'] = mf.trapmf(weight.universe, [45, 55, 55, 65])
weight['weight'] = mf.trapmf(weight.universe, [55, 75, 85, 120])

bmi['verythin'] = mf.trapmf(bmi.universe, [12, 12, 16, 17])
bmi['thin'] = mf.trapmf(bmi.universe, [16, 17, 17.5, 18.5])
bmi['normal'] = mf.trapmf(bmi.universe, [17.5, 18.5, 24, 25])
bmi['weight'] = mf.trapmf(bmi.universe, [24, 25, 26, 27])
bmi['obesity'] = mf.trapmf(bmi.universe, [26, 27, 32, 32])

In [6]:
#Rules
rule1 = ctrl.Rule(height['short'] & weight['thin'], bmi['normal'])
rule2 = ctrl.Rule(height['short'] & weight['normal'], bmi['weight'])
rule3 = ctrl.Rule(height['short'] & weight['weight'], bmi['obesity'])
rule4 = ctrl.Rule(height['normal'] & weight['thin'], bmi['thin'])
rule5 = ctrl.Rule(height['normal'] & weight['normal'], bmi['normal'])
rule6 = ctrl.Rule(height['normal'] & weight['weight'], bmi['weight'])
rule7 = ctrl.Rule(height['tall'] & weight['thin'], bmi['verythin'])
rule8 = ctrl.Rule(height['tall'] & weight['normal'], bmi['thin'])
rule9 = ctrl.Rule(height['tall'] & weight['weight'], bmi['normal'])

In [7]:
# Create control system
bmi_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9])
bmi_classification = ctrl.ControlSystemSimulation(bmi_ctrl)

In [8]:
# Classify BMI for each row in the DataFrame
df['BMI_Classification'] = np.nan

In [9]:
for index, row in df.iterrows():
    # Set input values
    bmi_classification.input['height'] = row['Height']
    bmi_classification.input['weight'] = row['Weight']

    # Compute the result
    bmi_classification.compute()

    # Assign classification to the DataFrame
    df.at[index, 'BMI_Classification'] = bmi_classification.output['bmi']

In [10]:
# Show the DataFrame with BMI classification
print(df[['Height', 'Weight', 'BMI_Classification']])

     Height  Weight  BMI_Classification
0       174      96           21.398525
1       189      87           21.250000
2       185     110           21.250000
3       195     104           21.250000
4       149      61           27.317960
..      ...     ...                 ...
495     150     153           28.690530
496     184     121           21.250000
497     141     136           28.690530
498     150      95           28.690530
499     173     131           21.552494

[500 rows x 3 columns]
